In [81]:
import numpy as np
import random
import secrets
import io
import scipy

N = 1000000
Alpha = 0.05

## Функція статистичних тестів

In [82]:
def test(seq: np.array, b = 8):
    n = len(seq)

    # Рівномірність
    counts = np.zeros(2**b)
    n_j = n / (2**b)

    for a in seq:
        counts[a] += 1

    stat_U = np.sum((counts - n_j)**2 / n_j)
    quant_U = scipy.stats.chi2.ppf(1 - Alpha, 2**b - 1)
    
    print("Тест на рівноймовірність символів:  " + str(stat_U <= quant_U) + "   (" + str(stat_U) + ")")


    # Незалежність символів
    pair_counts = np.zeros((2**b, 2**b), dtype=float)

    for i in range(0, n-1, 1):
        pair_counts[seq[i]][seq[i+1]] = pair_counts[seq[i]][seq[i+1]] + 1

    S = 0
    for i in range(0, 2**b):
        for j in range(0, 2**b):
            d = (np.sum(pair_counts[i, :]) * np.sum(pair_counts[:, j]))
            if d != 0:
                S += (pair_counts[i, j] ** 2) / d

    stat_I = n * (S - 1)
    quant_I = scipy.stats.chi2.ppf(1 - Alpha, (2**b - 1)**2)

    print("Тест на незалежність символів:  " + str(stat_I <= quant_I)  + "   (" + str(stat_I) + ")")


    # Однорідність послідовності
    r = 200
    interval_counts = np.zeros((r, 2**b), dtype=float)

    for i in range(0, r):
        for j in range(0, n // r):
            c = seq[r*i + j]
            interval_counts[i, c] += 1

    S = 0
    for i in range(0, r):
        for j in range(0, 2**b):
            d = (np.sum(interval_counts[i, :]) * np.sum(interval_counts[:, j]))
            if d != 0:
                S += (interval_counts[i, j] ** 2) / d
    
    stat_H = n * (S - 1)
    quant_H = scipy.stats.chi2.ppf(1 - Alpha, (2**b - 1) * (r - 1))

    print("Тест на однорідність послідовності:  " + str(stat_H <= quant_H)  + "   (" + str(stat_H) + ")")


### Генератор Лемера

In [83]:
class Linear_Low:
    a = 2**16 + 1
    c = 119
    x0 = 5  # тюряга #

    def generate_bytes(self, n: int):
        seq = np.zeros(n, dtype=np.uint32)
        seq[0] = self.x0

        for i in range(0, n - 1):
            seq[i+1] = (self.a*seq[i] + self.c) 

        seq = np.array(seq % (2**8), dtype=np.uint8)

        return seq

class Linear_High:
    a = 2**16 + 1
    c = 119
    x0 = 1  # тюряга #

    def generate_bytes(self, n: int):
        seq = np.zeros(n, dtype=np.uint32)
        seq[0] = self.x0

        for i in range(0, n - 1):
            seq[i+1] = (self.a*seq[i] + self.c)

        seq = np.array(seq >> 24, dtype=np.uint8)

        return seq


In [84]:
de1 = Linear_Low()
de2 = Linear_High()

t1 = de1.generate_bytes(N)
t2 = de2.generate_bytes(N)

test(t1)
print()
test(t2)

Тест на рівноймовірність символів:  True   (0.012288)
Тест на незалежність символів:  False   (255000000.0)
Тест на однорідність послідовності:  True   (652.771437011701)

Тест на рівноймовірність символів:  True   (23.938560000000003)
Тест на незалежність символів:  True   (63474.788106759435)
Тест на однорідність послідовності:  True   (45609.157861016494)


### Генератор L20

In [85]:
class L20:
    def __init__(self, x_init: np.array):
        self.x_init = x_init

    def generate_bits(self, n: int):
        seq = np.concatenate([np.array(self.x_init, dtype=np.uint8), np.zeros(n - 20, dtype=np.uint8)])

        for i in range(20, n):
            seq[i] = seq[i - 3] ^ seq[i - 5] ^ seq[i - 9] ^ seq[i - 20]

        return seq
            

In [86]:
smp = np.array([random.randint(0, 1) for _ in range(20)])
print(smp)
de_L20 = L20(smp)

test(de_L20.generate_bits(N), 1)

[1 0 0 1 0 1 0 1 1 1 1 1 0 0 1 1 0 0 0 0]
Тест на рівноймовірність символів:  True   (0.024964)
Тест на незалежність символів:  True   (0.01061416465830689)
Тест на однорідність послідовності:  True   (178.61557057496924)


### Генератор L89

In [104]:
class L89:
    def __init__(self, x_init: np.array):
        self.x_init = x_init

    def generate_bits(self, n: int):
        seq = np.concatenate([np.array(self.x_init, dtype=np.uint8), np.zeros(n - 89, dtype=np.uint8)])

        for i in range(89, n):
            seq[i] = seq[i - 38] ^ seq[i - 89]

        return seq
            

In [105]:
smp = np.array([random.randint(0, 1) for _ in range(89)])
de_L89 = L89(smp)

test(de_L89.generate_bits(100000), 1)

Тест на рівноймовірність символів:  True   (0.24336)
Тест на незалежність символів:  True   (1.6060988450306013)
Тест на однорідність послідовності:  True   (226.05617710460547)


### Генератор Вольфрама

In [89]:
# В ПІТОНІ НЕМА ВБУДОВАНОГО ЦИКЛІЧНОГО ЗСУВУ
def rcs(n: np.uint32, rotations) -> np.uint32: 
    return (n >> rotations | n << (32-rotations)) % 2**32

def lcs(n: np.uint32, rotations) -> np.uint32:
    return (n << rotations | n >> (32-rotations)) % 2**32

class Wolfram:
    def __init__(self, r0: np.uint32):
        self.r0 = r0

    def generate_bits(self, n: int):
        r_i = self.r0
        seq = np.zeros(n, dtype=np.uint8)

        for i in range(0, n):
            seq[i] = r_i % 2
            r_i = lcs(r_i, 1) ^ (r_i | rcs(r_i, 1))

        return seq
            

In [90]:
de_wolfram = Wolfram(1)

test(de_wolfram.generate_bits(N), 1)

Тест на рівноймовірність символів:  True   (0.391876)
Тест на незалежність символів:  True   (0.7214706982949792)
Тест на однорідність послідовності:  True   (218.66232493117366)


### Генератор "Бібліотекар"

In [91]:
class Librarian:
    def __init__(self, filename):
        file = io.open(filename, mode='r', encoding='utf-8')
        self.text = file.read()

    
    def generate_bytes(self, n: int):
        if len(self.text) < n:
            raise RuntimeError("Nema sliv, odni emotions")

        seq = np.zeros(n, dtype=np.uint8)

        for i in range(0, n):
            seq[i] = ord(self.text[i])

        return seq
            

In [92]:
de_Lb = Librarian("fanfiction.txt")

test(de_Lb.generate_bytes(N))

C:\Users\matki\AppData\Local\Temp\ipykernel_6032\3485778653.py:14: DeprecationWarning: NumPy will stop allowing conversion of out-of-bound Python integers to integer arrays.  The conversion of 9813 to uint8 will fail in the future.
For the old behavior, usually:
    np.array(value).astype(dtype)
will give the desired result (the cast overflows).
  seq[i] = ord(self.text[i])
C:\Users\matki\AppData\Local\Temp\ipykernel_6032\3485778653.py:14: DeprecationWarning: NumPy will stop allowing conversion of out-of-bound Python integers to integer arrays.  The conversion of 8230 to uint8 will fail in the future.
For the old behavior, usually:
    np.array(value).astype(dtype)
will give the desired result (the cast overflows).
  seq[i] = ord(self.text[i])
C:\Users\matki\AppData\Local\Temp\ipykernel_6032\3485778653.py:14: DeprecationWarning: NumPy will stop allowing conversion of out-of-bound Python integers to integer arrays.  The conversion of 8211 to uint8 will fail in the future.
For the old be

Тест на рівноймовірність символів:  False   (15618643.88096)
Тест на незалежність символів:  False   (4077664.325197495)
Тест на однорідність послідовності:  True   (33631.92420870798)


### Генератор BM

In [93]:
class BM:
    def __init__(self, p, a):
        self.p = p
        self.a = a

    def generate_bits(self, n: int):
        seq = np.zeros(n, dtype=object)
        seq[0] = random.randint(0, self.p - 1) 

        for i in range(1, n):
            seq[i] = pow(self.a, seq[i - 1], self.p)

        seq = np.array(seq < (self.p - 1) / 2, dtype=np.uint8) 

        return seq
    
    def generate_bytes(self, n: int):
        seq = np.zeros(n, dtype=object)
        seq[0] = random.randint(0, self.p - 1) 

        for i in range(1, n):
            seq[i] = pow(self.a, seq[i - 1], self.p)

        seq = np.array((seq * 256) // (self.p - 1), dtype=np.uint8) 

        return seq
            

In [94]:
p = int("CEA42B987C44FA642D80AD9F51F10457690DEF10C83D0BC1BCEE12FC3B6093E3", 16)
a = int("5B88C41246790891C095E2878880342E88C79974303BD0400B090FE38A688356", 16)

de_BM = BM(p, a)

print('Bits: ')
test(de_BM.generate_bits(N), 1)
print()
print('Bytes: ')
test(de_BM.generate_bytes(N))

Bits: 
Тест на рівноймовірність символів:  True   (0.010404)
Тест на незалежність символів:  True   (0.6288337717208492)
Тест на однорідність послідовності:  False   (244.73268045710483)

Bytes: 
Тест на рівноймовірність символів:  True   (247.483904)
Тест на незалежність символів:  True   (64275.34174213379)
Тест на однорідність послідовності:  True   (43710.68940246103)


### Генератор Джиффі

In [95]:
class Geffe:
    def __init__(self, x_init: np.array, y_init: np.array, s_init: np.array):
        self.x = x_init # 11 bits, x11 = x0 + x2
        self.y = y_init # 9 bits,  y9 = y0 + y1 + y3 + y4
        self.s = s_init # 10 bits, s10 = s0 + s3

    def generate_bits(self, n: int):
        seq = np.zeros(n, dtype=np.uint8)

        for i in range(0, n):
            seq[i] = (self.s[0] * self.x[0]) ^ ((1 ^ self.s[0]) * self.y[0])
            # Linear Shift
            self.x[0] = self.x[0] ^ self.x[2]
            self.x = np.roll(self.x, -1)
            self.y[0] = self.y[0] ^ self.y[1] ^ self.y[3] ^ self.y[4]
            self.y = np.roll(self.y, -1)
            self.s[0] = self.s[0] ^ self.s[3]
            self.s = np.roll(self.s, -1)

        return seq
            

In [96]:
x = np.array([random.randint(0, 1) for _ in range(11)])
y = np.array([random.randint(0, 1) for _ in range(9)])
s = np.array([random.randint(0, 1) for _ in range(10)])

print(x)
print(y)
print(s)

Ge_generator = Geffe(x, y, s)

test(Ge_generator.generate_bits(N), 1)

[0 1 1 1 0 0 0 1 1 0 1]
[0 1 0 1 1 1 0 1 0]
[0 1 1 0 0 1 0 1 0 1]
Тест на рівноймовірність символів:  True   (1.236544)
Тест на незалежність символів:  False   (5.402099677587202)
Тест на однорідність послідовності:  True   (70.42666095813388)


### Генератор BBS

In [97]:
class BBS:
    def __init__(self, p, q):
        self.n = p*q

    def generate_bits(self, n: int):
        seq = np.zeros(n, dtype=object)
        seq[0] = random.randint(2, self.n - 1) 

        for i in range(1, n):
            seq[i] = pow(seq[i - 1], 2, self.n)

        seq = np.array(seq % 2, dtype=np.uint8) 

        return seq
    
    def generate_bytes(self, n: int):
        seq = np.zeros(n, dtype=object)
        seq[0] = random.randint(2, self.n - 1) 

        for i in range(1, n):
            seq[i] = pow(seq[i - 1], 2, self.n)

        seq = np.array(seq % (2**8), dtype=np.uint8) 

        return seq
            

In [98]:
p = int("D5BBB96D30086EC484EBA3D7F9CAEB07", 16)
q = int("425D2B9BFDB25B9CF6C416CC6E37B59C1F", 16)

de_BBS = BBS(p, q)

print('Bytes: ')
test(de_BBS.generate_bytes(N))
print()
print('Bits: ')
test(de_BBS.generate_bits(N), 1)

Bytes: 
Тест на рівноймовірність символів:  True   (235.768832)
Тест на незалежність символів:  False   (65781.85683334059)
Тест на однорідність послідовності:  True   (43778.32293700079)

Bits: 
Тест на рівноймовірність символів:  True   (0.0441)
Тест на незалежність символів:  True   (0.00012004131022536058)
Тест на однорідність послідовності:  True   (71.42473196797283)


### Вбудовані генератори

#### Криптографічно нестійкий генератор

In [99]:
unsafe_bits_seq = np.array([random.randint(0, 1) for _ in range(N)], dtype=np.uint8)
test(unsafe_bits_seq, 1)

unsafe_bytes_seq = np.array(list(random.randbytes(N)), dtype=np.uint8)
test(unsafe_bytes_seq)

Тест на рівноймовірність символів:  True   (0.374544)
Тест на незалежність символів:  True   (0.01407232752548282)
Тест на однорідність послідовності:  True   (152.73429949180795)
Тест на рівноймовірність символів:  True   (278.22233600000004)
Тест на незалежність символів:  True   (65249.231660186924)
Тест на однорідність послідовності:  True   (45238.422695147885)


#### Криптографічно стійкий генератор

In [100]:
safe_bits_seq = np.array([secrets.randbelow(2) for _ in range(N)], dtype=np.uint8)
test(safe_bits_seq, 1)

safe_bytes_seq = np.array(list(secrets.token_bytes(N)), dtype=np.uint8)
test(safe_bytes_seq)

Тест на рівноймовірність символів:  False   (5.373124)
Тест на незалежність символів:  True   (0.01308031927216291)
Тест на однорідність послідовності:  True   (217.93548463611145)
Тест на рівноймовірність символів:  False   (309.073408)
Тест на незалежність символів:  True   (65001.953408504894)
Тест на однорідність послідовності:  True   (45173.44104688892)
